### Задание

В StackOverflow обсуждаются тысячи тем каждый день и используют столько слов и технологий! Вы задались резонным вопросом - какие слова, фразы используются наиболее часто за последние 6 месяцев? Попробуйте ответить на этот вопрос и пришлите решение ссылкой на GitHub, где будут код и графики / таблицы, отвечающие на этот вопрос.

В этом деле вам поможет StackOverflow Query, с помощью которого вы сможете получить необходимую вам выборку данных для анализа.

### Решение

Выгрузил тексты комментариев в csv-файл.

Поскольку на StackOverflow Query стоит ограничение в 50 000 строк, извлекаемых sql запросом, а данных гораздо больше, я выгружал комментарии, которые были опубликованы в период с 16 до 17 часов дня. Такая вот хитрость.

Получилось 158к комментариев. На основе них и проведен анализ.

In [1]:
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [2]:
stopwords = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

In [3]:
df = pd.read_csv('comments.csv')

df

,date,text
0,2021-09-15 16:00:06,What have you try?
1,2021-09-15 16:00:07,could you provide us a JSON?
2,2021-09-15 16:00:13,You're imposing validation in your markup. How...
3,2021-09-15 16:00:19,"Are you looking to edit the file itself, or to..."
4,2021-09-15 16:00:21,Use a private / incognito window or manually r...
...,...,...
158012,2022-03-12 16:59:21,"I'm not sure what the reason is, I assume it w..."
158013,2022-03-12 16:59:30,"Hi, I can give you a little course if you can ..."
158014,2022-03-12 16:59:31,Thank you it worked.
158015,2022-03-12 16:59:47,"Please don't post screenshots of your code, or..."


Токенизируем комментарии и составим словарь с количеством вхождений того или иного слова. Стоп-слова будем пропускать. В этом нам поможет библиотека nltk:

In [4]:
vocab = {}
token_count = 0

for comment in df.text:
    for word in tokenizer.tokenize(comment):
        word = word.lower()
        if word in stopwords:
            continue
        if word not in vocab:
            vocab[word] = 0
        
        vocab[word] +=1
        
        token_count +=1

Всего было встречено слов (не считая стопслов):

In [5]:
len(vocab)

114403

Всего было встречено токенов (не считая стопслов):

In [6]:
token_count

2420881

Отсортируем слова по их встречаемости:

In [7]:
word_freq_sorted = sorted(vocab.items(), key = lambda item : item[1], reverse=True)

Результат положим в DataFrame: 

In [8]:
words_df = pd.DataFrame(word_freq_sorted)
words_df = words_df.rename(columns={0:'word', 1:'count'})
words_df['frequency'] = words_df['count'] / token_count

Итак 50 наиболее часто встречающихся слов за последние полгода:

In [9]:
words_df.head(50)

,word,count,frequency
0,https,21573,0.008911
1,code,20001,0.008262
2,use,18512,0.007647
3,com,16942,0.006998
4,answer,16322,0.006742
5,question,15285,0.006314
6,using,13402,0.005536
7,thanks,12737,0.005261
8,would,12690,0.005242
9,like,12634,0.005219
